# Initiation of notebook


First get the repo from github to experiment on its models


we have 2 models to experiment on


```model_1```:
Denoising autoencoder + CNN 

```model_2```:
Encoder of the DAE + CNN

In [1]:

from google.colab import drive
drive.mount('/content/gdrive/') 


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd gdrive/My\ Drive/ColabFiles/AdversarialAttacks/

/content/gdrive/My Drive/ColabFiles/AdversarialAttacks


In [3]:
!git config --global user.email "jafer.ba@Gmail.com"

!git fetch --all

!git reset --hard origin/master


Fetching origin
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/JafarBadour/Adversarial-Attacks-on-Neural-Networks
   0530fd6..99dd08e  master     -> origin/master
HEAD is now at 99dd08e feat: added ```generate_adversarial_data``` given a model and an attack criterion generate adversarial examples from the file adversarial_utils.py


In [4]:
ls Models/ -la

total 8850
drwx------ 2 root root    4096 Apr 25 18:23 DAE/
-rw------- 1 root root  105216 Apr 25 18:23 DAE.h
-rw------- 1 root root  105304 Apr 25 19:31 DAE.h5
-rw------- 1 root root   17168 Apr 25 19:31 decoder.h5
-rw------- 1 root root 4320248 Apr 25 19:31 denoising_autoencoder_ann.h5
-rw------- 1 root root 4465816 Apr 25 19:31 encoder_ann.h5
-rw------- 1 root root   43176 Apr 25 19:31 encoder.h5


In [5]:

!pip install -r requirements.txt

# RUN ATTACKS


on these two models



In [3]:
import numpy as np
import matplotlib.pyplot as plt
# %tensorflow_version 1.x

import tensorflow as tf
from tensorflow import keras

In [4]:
tf.__version__

'2.1.0'

In [8]:

import foolbox

# Import DAE_ANN

DAE_ANN = keras.models.load_model('./Models/denoising_autoencoder_ann.h5')

fmodel = foolbox.models.TensorFlowModel.from_keras(DAE_ANN, bounds=(0, 1))
attack_criterion = foolbox.criteria.Misclassification()
attack_fn = foolbox.attacks.GradientSignAttack(fmodel, criterion=attack_criterion, distance=foolbox.distances.Linfinity)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.





In [9]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
train_images = train_images/255.0
test_images = test_images/255.0
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)
test_loss, test_acc = DAE_ANN.evaluate(test_images, test_labels)
print('Model accuracy based on test data:', test_acc)
del fashion_mnist

10000/10000 [==============================] - 2s 224us/sample - loss: 0.3759 - acc: 0.8662
Model accuracy based on test data: 0.8662


In [13]:
from utils.adversarial_utils import generate_adversarial_data
x_images = train_images[0:6000, :]
print(x_images.shape)
(x_train_adv_images, x_train_adv_perturbs, x_train_labels) = \
generate_adversarial_data(original_images = x_images, predictions = DAE_ANN.predict(x_images), attack_fn = attack_fn)


(6000, 28, 28, 1)


ValueError: ignored